In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data"))

# Any results you write to the current directory are saved as output.

['histopathologic-cancer-detection.zip', 'sample_submission.csv', 'test', 'train', 'train_labels.csv']


In [2]:
#Loading data 

train_dir = "../data/train/"
test_dir = "../data/test"

df_train = pd.read_csv('../data/train_labels.csv',dtype=str)
df_train.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [3]:
type(df_train.id)

pandas.core.series.Series

In [4]:
df_test=pd.read_csv("../data/sample_submission.csv",dtype=str)

# add extension to image filenames 
def append_ext(fn): 
    return fn+".tif"
df_train["id"]=df_train["id"].apply(append_ext)
df_test["id"]=df_test["id"].apply(append_ext)

print(df_train.head())

                                             id label
0  f38a6374c348f90b587e046aac6079959adf3835.tif     0
1  c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif     1
2  755db6279dae599ebb4d39a9123cce439965282d.tif     0
3  bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif     0
4  068aba587a4950175d04c680d38943fd488d6a9d.tif     0


In [5]:
type(df_train.id[1])

str

In [6]:
from sklearn.model_selection import train_test_split

# df = df_train.sample(n=10000, random_state=2018)
df = df_train # using full dataset
#train, valid = train_test_split(df,test_size=0.2)

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  validation_split=0.2)

test_datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x)

In [13]:
# use flow_from_dataframe method to build train and valid generator
# Only shuffle the train generator as we want valid generator to have the same structure as test

train_generator = train_datagen.flow_from_directory(
                '../data/train/',
                target_size=(96, 96),
                classes=['0', '1'],
                batch_size=64,
                shuffle=True,    
                subset='training',
                class_mode='binary'
                )

valid_generator = train_datagen.flow_from_directory(
                'C:/Users/elisa/PycharmProjects/kaggle_histopathologic_cancer/data/train/',
                target_size=(96, 96),
                classes=['0', '1'],
                batch_size=64,
                shuffle=False,    
                subset='validation',
                class_mode='binary'
                )



Found 176021 images belonging to 2 classes.
Found 44004 images belonging to 2 classes.


In [9]:
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras import layers

IMG_SIZE = (96, 96)
IN_SHAPE = (*IMG_SIZE, 3)

dropout_dense=0.5

conv_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)
       
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(dropout_dense))
model.add(layers.Dense(1, activation = "sigmoid"))

conv_base.summary()

94773248/94765736 [==============================] - 27s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 48, 48, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 48, 48, 64)   256         conv1_conv[0][0]                 
_________________________

In [24]:
# freeze layer. unfreeze start at layer conv2_block1_1_conv. if freeze everything, val acc will be really bad

conv_base.Trainable=True

set_trainable=False
for layer in conv_base.layers:
    if layer.name == 'conv2_block1_1_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [25]:
from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.01), loss = "binary_crossentropy", metrics=["accuracy"])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.1)

history = model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=8,
                   callbacks=[reducel, earlystopper])

Epoch 1/8
2278/2750 [=======================>......] - ETA: 1:05:05 - loss: 0.3804 - accuracy: 0.8260

In [ ]:
# predict test and submission
from glob import glob
from skimage.io import imread

base_test_dir = '../input/test/'
test_files = glob(os.path.join(base_test_dir,'*.tif'))
submission = pd.DataFrame()
file_batch = 5000
max_idx = len(test_files)
for idx in range(0, max_idx, file_batch):
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})
    test_df['id'] = test_df.path.map(lambda x: x.split('/')[3].split(".")[0])
    test_df['image'] = test_df['path'].map(imread)
    K_test = np.stack(test_df["image"].values)
    K_test = (K_test - K_test.mean()) / K_test.std()
    predictions = model.predict(K_test)
    test_df['label'] = predictions
    submission = pd.concat([submission, test_df[["id", "label"]]])
submission.head()

In [ ]:
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d%m%Y_%H%M%S")
print("date and time =", dt_string)


In [ ]:
submission.to_csv("../submission/submission_"+dt_string+".csv", index = False, header = True)